In [26]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Output')
cur_date = "122322"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
# need to make some of the cols numeric
acs_df[,4:30] = apply(acs_df[,4:30], 2, as.numeric)
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_121222.xlsx", sheet = 2))

Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric

In [27]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,20.8,14.6,26.4,28.0,54.8,0,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,33.0,19.2,27.3,37.8,44.6,0,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,17.3,22.0,18.0,28.3,21.0,NA,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,26.8,14.0,30.0,33.3,19.9,NA,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,12.2,7.1,12.3,14.8,18.9,0,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,22.3,19.2,17.7,31.0,27.8,100,14.2,19.5,14.7,25.0


,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,14000US37001020100,201.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
2,14000US37001020200,202.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
3,14000US37001020301,203.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%
4,14000US37001020302,203.02,Alamance,278233.79700000002,92.237034639041696,1,0-20%
5,14000US37001020400,204.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
6,14000US37001020501,205.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%


In [28]:
# seeing if the census tracts are consistent each file
length(unique(acs_df$Census_Tract))
length(unique(wildfire_hazard_df$Census_Tract))

[1] 1410

[1] 1776

In [29]:
# they're not so what tracts aren't consistent?
`%notin%` <- Negate(`%in%`)
wildfire_hazard_df %>%
    filter(Census_Tract %notin% unique(acs_df$Census_Tract)) %>%
    select(c("GEO_ID", "Census_Tract", "County")) 

GEO_ID,Census_Tract,County
<chr>,<dbl>,<chr>
14000US37007920302,9203.02,Anson
14000US37007980000,9800.00,Anson
14000US37019020107,201.07,Brunswick
14000US37019020108,201.08,Brunswick
14000US37019020205,202.05,Brunswick
14000US37019020206,202.06,Brunswick
14000US37019020406,204.06,Brunswick
14000US37019020407,204.07,Brunswick
14000US37019020408,204.08,Brunswick


In [30]:
# lot's of missing data
dim(full_join(acs_df, wildfire_hazard_df))
dim(inner_join(acs_df, wildfire_hazard_df))

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 3137   34

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 1730   34

In [31]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

# need to make some of the cols numeric
acs_whp_df[,30:33] = apply(acs_whp_df[,30:33], 2, as.numeric)
head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”


,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,NA,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


# Random Forest Imputation
Generating missing values using random forest (RF).

In [53]:
test = acs_whp_df$Poverty_Asian
test

test == "NA"

[1]   0.0   0.0    NA   0.0 100.0   0.0   0.0   0.0    NA   0.0  23.4   0.0
  [13]    NA   0.0   0.0  61.5 100.0    NA   0.0    NA   0.0   0.0    NA    NA
  [25]   0.0   0.0  42.5   8.6    NA   0.0    NA   0.0    NA   0.0    NA    NA
  [37]    NA    NA  16.3    NA  43.9  54.5   0.0  11.1    NA    NA  39.7    NA
  [49]   0.0    NA    NA   3.9  42.9   0.0    NA 100.0   0.0    NA    NA    NA
  [61]   0.0    NA 100.0   0.0   0.0    NA   0.0   0.0   0.0    NA   0.0 100.0
  [73]  15.1    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA   0.0
  [85]   0.0   0.0    NA   0.0    NA    NA    NA    NA  59.1   0.0  17.5   0.0
  [97]   0.0    NA  29.6    NA    NA  16.4  14.5    NA   0.0   0.0   0.0   0.0
 [109]   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   9.3 100.0   0.0    NA
 [121]    NA   0.0    NA    NA    NA   0.0   0.0   0.0   0.0    NA    NA   0.0
 [133]    NA   0.0  75.0   0.0    NA    NA    NA    NA   0.0   4.8   0.0    NA
 [145]  31.3   0.0   5.2   7.4   0.0   0.0  20.0    NA    NA  27.0    NA    NA
 [157]    NA    NA   0.0    NA   0.0  20.1   0.0   0.0   0.0    NA    NA   0.0
 [169]  80.0 100.0   0.0   0.0   0.0   0.0    NA   0.0   0.0   0.0   0.0   0.0
 [181]   0.0    NA    NA    NA   0.0   0.0   0.0   0.0 100.0    NA    NA    NA
 [193]    NA   0.0   0.0    NA   0.0   0.0    NA 100.0    NA    NA    NA    NA
 [205]  41.5 100.0    NA  33.3    NA   0.0   0.0    NA   0.0    NA    NA   0.0
 [217]   0.0   0.0   0.0   0.0   0.0    NA    NA    NA    NA   0.0   0.0   0.0
 [229]   0.0   0.0    NA   0.0  31.0  14.1   0.0   0.0  36.8   9.2  66.4   5.8
 [241]  84.9    NA   8.7   0.0   0.0  55.6   5.7    NA   0.0  44.0   0.0   0.0
 [253]    NA   0.0   0.0  20.0   6.9   0.0   0.0   9.1   0.0   0.0    NA   0.0
 [265]    NA    NA   0.0   0.0    NA 100.0  34.8    NA   0.0    NA   0.0   0.0
 [277]   0.0   0.0    NA   0.0   0.0   0.0    NA    NA  46.4   0.0    NA   0.0
 [289]    NA   0.0   0.0    NA   0.0   0.0   0.0    NA    NA    NA    NA   0.0
 [301]  29.3   0.0    NA   0.0    NA   0.0  39.2    NA   0.0  62.2  40.6    NA
 [313]   0.0   0.0   0.0   0.0   0.0   0.0    NA   0.0   0.0  23.4   0.0   0.0
 [325]  30.0  31.9  47.7 100.0   0.0    NA    NA   4.8   0.0   0.0  28.8  25.6
 [337]   0.0   0.0  17.1   0.0    NA  77.8  45.3  40.4   0.0   0.0    NA    NA
 [349]   7.7   0.0  16.0   0.0   0.0   4.4  31.6  24.6   0.0  16.7  10.6   0.0
 [361]   0.0  50.0    NA  35.0   0.0   0.0    NA    NA   0.0   0.0   0.0   0.0
 [373]  66.7    NA   0.0    NA    NA    NA    NA    NA   0.0    NA    NA    NA
 [385]   4.2   0.0    NA    NA    NA   0.0   0.0   0.0   0.0   0.0   0.0    NA
 [397]    NA   0.0   0.0   0.0    NA    NA    NA   0.0   0.0    NA   0.0    NA
 [409]   0.0   0.0   0.0   0.0    NA   0.0    NA    NA    NA  10.1   0.0  15.3
 [421]   8.3   0.0  11.2  13.2  36.9  17.1   0.0   0.0  89.2    NA 100.0    NA
 [433]    NA    NA 100.0    NA   0.0   0.0   0.0   0.0  20.2   0.0   0.0   0.0
 [445]  85.4    NA   5.0    NA   3.2   0.0   0.0   0.0   0.0    NA   9.3  28.0
 [457]  35.4   3.9   5.9  34.7   3.7  23.4   0.0   0.0  42.9  41.3   0.0    NA
 [469]   0.0   0.0    NA    NA    NA    NA    NA   0.0    NA    NA    NA    NA
 [481]    NA  15.0  38.0   0.0    NA    NA    NA    NA    NA    NA    NA    NA
 [493]   6.0  21.7   0.0   0.0  45.2    NA   0.0   0.0    NA   0.0    NA   0.0
 [505]   0.0   0.0   0.0  25.9  12.7   0.0   0.0   0.0   0.0    NA   0.0   0.0
 [517]   0.0   0.0    NA 100.0  46.2  67.2    NA    NA    NA   0.0   0.0  10.2
 [529]   0.0   0.0   0.0  38.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0  11.1
 [541] 100.0    NA   0.0   0.0   0.0   0.0   0.0   0.0  12.1    NA   0.0   0.0
 [553]   4.6   0.0  12.1   0.0   0.0   0.0   0.0  24.2  33.0   0.0   0.3   0.0
 [565]   0.0    NA  20.8   0.0   0.0   0.0   0.0  22.3   0.0    NA 100.0   0.0
 [577]    NA    NA   0.0   0.0    NA    NA   0.0   0.0  12.5    NA    NA    NA
 [589]  20.0    NA    NA    NA   0.0   0.0   0.0   0.0   0.0    NA    NA   0.0
 [601]   0.0 100.0   0.0    NA    NA    NA    NA    N

[1] FALSE FALSE    NA FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE FALSE
  [13]    NA FALSE FALSE FALSE FALSE    NA FALSE    NA FALSE FALSE    NA    NA
  [25] FALSE FALSE FALSE FALSE    NA FALSE    NA FALSE    NA FALSE    NA    NA
  [37]    NA    NA FALSE    NA FALSE FALSE FALSE FALSE    NA    NA FALSE    NA
  [49] FALSE    NA    NA FALSE FALSE FALSE    NA FALSE FALSE    NA    NA    NA
  [61] FALSE    NA FALSE FALSE FALSE    NA FALSE FALSE FALSE    NA FALSE FALSE
  [73] FALSE    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA FALSE
  [85] FALSE FALSE    NA FALSE    NA    NA    NA    NA FALSE FALSE FALSE FALSE
  [97] FALSE    NA FALSE    NA    NA FALSE FALSE    NA FALSE FALSE FALSE FALSE
 [109] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA
 [121]    NA FALSE    NA    NA    NA FALSE FALSE FALSE FALSE    NA    NA FALSE
 [133]    NA FALSE FALSE FALSE    NA    NA    NA    NA FALSE FALSE FALSE    NA
 [145] FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA    NA FALSE    NA    NA
 [157]    NA    NA FALSE    NA FALSE FALSE FALSE FALSE FALSE    NA    NA FALSE
 [169] FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE FALSE FALSE FALSE
 [181] FALSE    NA    NA    NA FALSE FALSE FALSE FALSE FALSE    NA    NA    NA
 [193]    NA FALSE FALSE    NA FALSE FALSE    NA FALSE    NA    NA    NA    NA
 [205] FALSE FALSE    NA FALSE    NA FALSE FALSE    NA FALSE    NA    NA FALSE
 [217] FALSE FALSE FALSE FALSE FALSE    NA    NA    NA    NA FALSE FALSE FALSE
 [229] FALSE FALSE    NA FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [241] FALSE    NA FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE FALSE FALSE
 [253]    NA FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE
 [265]    NA    NA FALSE FALSE    NA FALSE FALSE    NA FALSE    NA FALSE FALSE
 [277] FALSE FALSE    NA FALSE FALSE FALSE    NA    NA FALSE FALSE    NA FALSE
 [289]    NA FALSE FALSE    NA FALSE FALSE FALSE    NA    NA    NA    NA FALSE
 [301] FALSE FALSE    NA FALSE    NA FALSE FALSE    NA FALSE FALSE FALSE    NA
 [313] FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE FALSE FALSE FALSE
 [325] FALSE FALSE FALSE FALSE FALSE    NA    NA FALSE FALSE FALSE FALSE FALSE
 [337] FALSE FALSE FALSE FALSE    NA FALSE FALSE FALSE FALSE FALSE    NA    NA
 [349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [361] FALSE FALSE    NA FALSE FALSE FALSE    NA    NA FALSE FALSE FALSE FALSE
 [373] FALSE    NA FALSE    NA    NA    NA    NA    NA FALSE    NA    NA    NA
 [385] FALSE FALSE    NA    NA    NA FALSE FALSE FALSE FALSE FALSE FALSE    NA
 [397]    NA FALSE FALSE FALSE    NA    NA    NA FALSE FALSE    NA FALSE    NA
 [409] FALSE FALSE FALSE FALSE    NA FALSE    NA    NA    NA FALSE FALSE FALSE
 [421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE    NA
 [433]    NA    NA FALSE    NA FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [445] FALSE    NA FALSE    NA FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE
 [457] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA
 [469] FALSE FALSE    NA    NA    NA    NA    NA FALSE    NA    NA    NA    NA
 [481]    NA FALSE FALSE FALSE    NA    NA    NA    NA    NA    NA    NA    NA
 [493] FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE    NA FALSE    NA FALSE
 [505] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE
 [517] FALSE FALSE    NA FALSE FALSE FALSE    NA    NA    NA FALSE FALSE FALSE
 [529] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [541] FALSE    NA FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE
 [553] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [565] FALSE    NA FALSE FALSE FALSE FALSE FALSE FALSE FALSE    NA FALSE FALSE
 [577]    NA    NA FALSE FALSE    NA    NA FALSE FALSE FALSE    NA    NA    NA
 [589] FALSE    NA    NA    NA FALSE FALSE FALSE FALSE FALSE    NA    NA FALSE
 [601] FALSE FALSE FALSE    NA    NA    NA    NA    N

In [48]:
a = acs_whp_df %>% 
filter(County == "Anson")

# test
a[a == NA] = " "
matrix_a = as.matrix(a[,4:32])
head(matrix_a)

#missForest(matrix_a)

Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,Race_Indigenous,Race_Asian,Race_Other,⋯,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean
94.5,39.9,24.0,76.0,17.1,47.1,48.4,0.0,0.0,0.0,⋯,19.6,19.8,23.6,NA,NA,100.0,NA,8.1,325443.8,528.9904
103.4,44.0,20.2,79.8,16.4,46.7,49.2,0.7,3.3,0.0,⋯,14.5,21.1,17.1,39.7,NA,NA,9.3,16.4,303328.1,408.4917
71.0,30.0,28.2,71.8,11.1,17.2,82.5,0.0,0.0,0.0,⋯,31.4,42.2,42.0,NA,NA,0.0,NA,17.6,343739.9,353.8732
136.9,42.9,18.3,81.7,22.1,48.6,44.9,0.0,1.9,3.1,⋯,9.5,10.8,7.3,0.0,0,29.1,10.0,12.5,353409.6,396.6235


In [18]:
# imputing sociodemographic data using RF
random_forest_imputation = function(dataset){
    
    imputed_df = data.frame()
    # iterating through each df grouped by county and variable
    for (i in 1:length(unique(dataset$County))){ 
        filtered_df = dataset %>%
            filter(County == unique(dataset$County)[i])
        print(unique(dataset$County)[i])
        imputed_RF_object = missForest(as.matrix(filtered_df[,4:32]))
        imputed_RF_df = imputed_RF_object$ximp
        imputed_values_df = data.frame(cbind(filtered_df[,c(1:3,33,34)], imputed_RF_df)) 

        # saving imputed values
        #imputed_df = rbind(imputed_df, imputed_values_df)

    }
    
    #return(imputed_final_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df)
head(imputed_df)

[1] "Alamance"
[1] "Alexander"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

[1] "Alleghany"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”


[1] "Anson"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”


ERROR: Error in if (n == 0) stop("data (x) has 0 rows"): argument is of length zero


In [9]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_ACS_WHP_Data.csv"), row.names = FALSE)